In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../tools"))
from envvars import load_env_vars_from_directory
import numpy as np
import pandas as pd
from datetime import datetime
from urllib.parse import urljoin

In [ ]:
# Variables de entorno
# Cargar variables de entorno desde archivos .env
load_env_vars_from_directory("../.env")

# Comprobar si las variables de entorno necesarias están definidas

cal_file =os.getenv('PODCAST_CAL_FILE')
if not cal_file:
    raise ValueError("PODCAST_CAL_FILE environment variable is not set.")
if not os.path.exists(cal_file):
    raise FileNotFoundError(f"Calendar '{cal_file}' does not exist.")

prefix = os.getenv('PODCAST_PREFIX')
if not prefix:
    raise ValueError("PODCAST_PREFIX environment variable is not set.")
print(f"Calendario de podcasts: {cal_file}")
print(f"Prefijo de podcasts: {prefix}")

In [ ]:
df = pd.read_csv(cal_file, parse_dates=["date"], index_col="episode" )
df['month'] = df['date'].dt.month
df['weekday'] = df['date'].dt.dayofweek
df['day'] = df['date'].dt.day
df['ep_ordinal'] = df['date'].apply(lambda x: x.toordinal())
print(df.info())

In [ ]:
INTERESTING_YEARS = 3
EPN = 550

In [ ]:
# Obtenemos los episodios interesantes, que son los que están dentro de los últimos INTERESTING_YEARS años
# y que son anteriores o iguales al episodio EPN
int_df = df[df.index <= EPN].copy()
int_df = int_df[int_df['date'] >= int_df['date'].max() - pd.DateOffset(years=INTERESTING_YEARS)]
int_df

In [ ]:
# Comprobar cuál es el día de la semana más frecuente
most_common_weekday = int_df['weekday'].mode()[0]
# Comprobar si la frecuencia de la moda es significativa
weekday_counts = int_df['weekday'].value_counts()
if weekday_counts[most_common_weekday] > 0.5 * len(int_df):
    print(f"El día de la semana más frecuente es {most_common_weekday} con {weekday_counts[most_common_weekday]} episodios.")
    print(f"La frecuencia de la moda es {weekday_counts[most_common_weekday] / len(int_df) * 100:.2f}% de los episodios.")
else:
    print("No hay un día de la semana más frecuente significativo.")

In [ ]:
# Cálculo del periodo de tiempo entre episodios
diffs = int_df['ep_ordinal'].diff() / int_df.index.to_series().diff()
diffs_mode = diffs.value_counts().idxmax()
diffs_mode_freq = diffs.value_counts()[diffs_mode]/ len(diffs)
print(f"El periodo de tiempo más frecuente entre episodios es {diffs_mode} días, con una frecuencia de {diffs_mode_freq * 100:.2f}%.")
if diffs_mode_freq > 0.8:
    period = diffs_mode
else:
    period = diffs.mean()



In [ ]:
# Cálculo de los meses inactivos
month_counts = int_df['month'].value_counts(normalize=True)
inactive_months = month_counts[month_counts < 0.05]
print(month_counts, inactive_months.index)

In [ ]:
last_episode = int_df.loc[int_df.index.max()]
# Calculamos el valor absoluto entre el día de la semana del último episodio y el día de la semana más frecuente
diff_weekday = most_common_weekday - last_episode['weekday']
diff_weekday
last_theoretical_date = last_episode['date'] + pd.DateOffset(days=diff_weekday)
print(f"Último episodio: {last_episode['date']} (Día de la semana: {last_episode['weekday']})")
print(f"Último episodio teórico: {last_theoretical_date} (Día de la semana: {most_common_weekday})")    

In [ ]:

int_df.index.max()

In [ ]:
if EPN > int_df.index.max():
    ld = last_theoretical_date
    for i in range(int_df.index.max() + 1, EPN + 1):
        ld += pd.DateOffset(days=period)
        if ld.month in inactive_months.index:
            print(f"Mes inactivo: {ld.month} - Episodio {i} no se añadirá.")
            continue
    offset = most_common_weekday - ld.weekday()
    ld += pd.DateOffset(days=offset)
    print(f"Próximo episodio teórico: {ld} (Día de la semana: {most_common_weekday})")    

In [ ]:
int_df.index.diffs()